# Build the Neural Network

이미지 분류 신경망 구성

In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

GPU/MPS 사용 여부 확인

In [4]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


클래스 정의

In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [6]:
# NeuralNetwork의 인스턴스(instance)를 생성하고 이를 device 로 이동한 뒤, 구조(structure)를 출력합니다.
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [25]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([9])


모델 계층(Layer)

In [35]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


nn.Flatten

In [39]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


nn,Linear

In [48]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


nn.ReLU

In [42]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.5599,  0.0271, -0.0705,  0.2922,  0.3903, -0.7977,  0.0180, -0.0858,
          0.4950,  0.2722,  0.1642, -0.7782, -0.0729,  0.2497, -0.0886,  0.4176,
         -0.2043,  0.0661, -0.3362, -0.1042],
        [-0.5491,  0.1008, -0.1365,  0.4532,  0.5030, -0.4053, -0.3404,  0.0603,
          0.4877,  0.2765,  0.1458, -0.5701, -0.0468,  0.1540, -0.3417,  0.3658,
         -0.1079,  0.3587, -0.2922, -0.1145],
        [-0.5685, -0.1870,  0.0575,  0.3405,  0.2914, -0.5718, -0.0820,  0.0895,
          0.4752,  0.1518,  0.1349, -0.6903, -0.0497,  0.0047, -0.4719,  0.1591,
         -0.2400,  0.1874, -0.1769,  0.1504]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0271, 0.0000, 0.2922, 0.3903, 0.0000, 0.0180, 0.0000, 0.4950,
         0.2722, 0.1642, 0.0000, 0.0000, 0.2497, 0.0000, 0.4176, 0.0000, 0.0661,
         0.0000, 0.0000],
        [0.0000, 0.1008, 0.0000, 0.4532, 0.5030, 0.0000, 0.0000, 0.0603, 0.4877,
         0.2765, 0.1458, 0.0000, 0.0000, 0.1540, 0.00

nn.Sequential

In [43]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

nn.Softmax

In [44]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

모델 매개변수

In [46]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0149, -0.0184,  0.0059,  ...,  0.0110, -0.0003, -0.0120],
        [ 0.0334, -0.0017, -0.0086,  ..., -0.0032,  0.0097, -0.0122]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0339, -0.0323], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0383, -0.0230,  0.0085,  ..., -0.0252, -0.0375,  0.0031],
        [ 0.0394,  0.0284, -0.0256,  ...,  0.0035, -0.0124,  0.0022]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | 